In [12]:
using LinearAlgebra, CairoMakie, BenchmarkTools

In [10]:
struct FitResult
    params
    errors
end

"""
    nFit

Levenberg-Marquardt curve−fitting : minimize sum of weighted squared residuals

- f = function of n-independent variables , `x` , and $m$ coefficients ,`p`, returning the simulated model : yhat = f(x, p)
- p0 = initial guess of coefficient values ( n x 1)
- xarr = arrays of independent variables (used as arg to func) (m x 1)
- yarr =  data t o be fit by f(x, p) (m x 1)
- earr =  nothing or Vector. weights or a scalar weight value. each entry should be positive (m x 1)
        inverse of the standa r d measurement errors
        Default :  (1/(yarr' ∗ yarr))

- MaxIter = maximum number of iteration.
- Lup, Ldown :
- ϵ1, ϵ2, ϵ3, ϵ4 :
- δ :  
"""
mutable struct nFit
    f::Function
    p::AbstractVector{<:Real}
    xarr::AbstractVector{<:Real}
    yarr::AbstractVector{<:Real}
    earr::Union{Nothing, AbstractVector{<:Real}}
    J::Matrix{<:Real}
    W::Union{Nothing, AbstractMatrix{<:Real}}
    MaxIter::Int
    λ::Real
    Lup::Real
    Ldown::Real
    ϵ1::Real
    ϵ2::Real
    ϵ3::Real
    ϵ4::Real
    δ::Real
    
    function nFit(f, p0, xarr, yarr, earr=nothing; MaxIter = 1000, λ=1.0e-2, Lup=11.0, Ldown=9.9, ϵ1 = 1.0e-3, ϵ2=1.0e-3, ϵ3 = 1.0e-1, ϵ4=1.0e-1, δ = 1.0e-3)
        @assert length(xarr) == length(yarr)
        if earr != nothing 
            @assert length(xarr) == length(earr)
        end
        try
            f(xarr[1], p0)
        catch
            error("Unexpected behavior of function and parameters")
        end
        J0 = zeros( length(xarr), length(p))
        W = (earr === nothing) ? nothing : Matrix(Diagonal(earr))

        new(f, p0, xarr, yarr, earr, J0, W, MaxIter, λ, Lup, Ldown, ϵ1, ϵ2, ϵ3, ϵ4, δ )
    end
end


Base.Meta.ParseError: ParseError:
# Error @ /Users/jiyongso/Documents/Projects/JuliaNumerical/src/topics/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W1sZmlsZQ==.jl:11:58

- f = function of n-independent variables , `x` , and $m$ coefficients ,`p`, returning the simulated model : yhat = f(x, p)
#                                                        └ ── identifier or parenthesized expression expected after $ in string

In [3]:

function jacobian(lm::nFit)
    J = Array{Float64}(undef, (length(lm.xarr), length(lm.p)))
    @inbounds for j ∈ 1:length(lm.p), i ∈ 1:length(lm.xarr)
        p1, p2 = lm.p[:], lm.p[:]
        p1[j] += lm.δ
        p2[j] -= lm.δ
        J[i, j] = (lm.f(lm.xarr[i], p1) - lm.f(lm.xarr[i], p2))/(2*lm.δ)
    end
    return J
end



function err(x::AbstractVector{<:Real}, y::AbstractVector{<:Real}, f::Function, p::AbstractVector)
    h(x) = f(x, p)
    return sum((y .- h.(x)).^2)
end

function err(lm::nFit)
    return err(lm.xarr, lm.yarr, lm.f, lm.p)
end

function update_params(lm::nFit, p)
    lm.p = p
end

function calc_result(lm::nFit)
    reducederr = err(lm::nFit)/(length(lm.xarr) - length(lm.p))
    cov = inv((lm.J)' * (lm.J))
    err = sqrt.(diag(cov))
end

function fit(lm::nFit)
    Nitter = 0
    ee = err(lm)
    
    qs = 0.0
    
    while Nitter < lm.MaxIter

        Nitter += 1
        J = jacobian(lm)
        JtJ = J'*J
        dD = Diagonal(diag(JtJ))
        dy =  lm.yarr .- [lm.f(t, lm.p) for t in lm.xarr]
        h = inv(JtJ .+ lm.λ .* dD) * J' * dy
        err0 = err(lm)
        # 수렴 확인
        cc1 = maximum(abs.(J'*dy)) < lm.ϵ1
        cc2 = maximum(abs.(lm.p ./ h)) < lm.ϵ2
        cc3 = err0/(length(lm.xarr) - length(lm.p)) < lm.ϵ3

        if cc1 || cc2 || cc3
            break
        end

        dn = (h' * (lm.λ .* dD) * h) .+ J' * dy
        ρ = err0 - err(lm.xarr, lm.yarr, lm.f, lm.p .+ h)/norm(dn) 


        if ρ > lm.ϵ4 
            lm.p = lm.p .+ h
            lm.λ = max(lm.λ/lm.Ldown, 1.0e-7)
        else 
            lm.λ = min(lm.λ*lm.Lup, 1.0e7)
        end
        println(lm.p, lm.λ, Nitter, err(lm))
    end
    return lm.p
end

fit (generic function with 1 method)

In [4]:
q(x, p) = p[1]*sin(x) + p[2] * x + p[3]
t=0.0:0.1:10.0
data = 5.0*sin.(t) .+ 7.0 .* t .+ (0.5.*rand(length(t)))

# jacobian(q, t, [2.0, 1], 0.001)
lm = nFit(q, [2.0, 1.0, 5.0], t, data, MaxIter=10)

nFit(Main.q, [2.0, 1.0, 5.0], 0.0:0.1:10.0, [0.42145300289609366, 1.2082340795267488, 2.821540016470666, 3.5848347229937314, 5.1314105050709635, 6.305999581006898, 7.402236435302888, 8.544458507748743, 9.338809569115645, 10.35313956368236  …  65.61910168086426, 65.9636471019667, 65.81469325414085, 66.35147884852174, 66.39089656063192, 66.44197896097994, 66.90015878324546, 66.89375563617027, 67.19498893154434, 67.30254159624315], nothing, [0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0], nothing, 10, 0.01, 11.0, 9.9, 0.001, 0.001, 0.1, 0.1, 0.001)

In [5]:
fit(lm)

[4.880797871246246, 6.743153638084552, 1.6012220134474977]0.00101010101010101154.94613973299553
[5.006905489139682, 6.989926440343225, 0.3120686973816773]0.0001020304050607080922.015622697933188


3-element Vector{Float64}:
 5.006905489139682
 6.989926440343225
 0.3120686973816773

In [7]:
err(lm)

2.015622697933188

In [ ]:
err(lm)